# Mediator
> Define an object that encapsulates how a set of objects interact. Mediator promotes loose coupling by keeping objects from referring to each other explicitly, and it lets you vary their interaction independently.

Instead of having a bunch of different objects all knowing about each others' types and methods (thereby being tightly coupled), insert a "mediator" object which has all this messy knowledge, but all the other objects are now only tightly coupled with this single mediator interface.

## Problem
Objects that are too tightly coupled are difficult to implement, change, test, and reuse. Moreover, the interaction is "hard coded" into the way the objects call each other, which makes the interaction flow itself hard to change. An extreme example of two tightly coupled objects are when they have a circular depedency as shown in the code below.

In [4]:
from typing import Optional

class BuddyOne:
    def __init__(self):
        self._buddy: Optional["BuddyTwo"] = None
        
    def set_buddy(self, buddy: "BuddyTwo"):
        self._buddy = buddy
        
    def action_one(self, arg):
        print(f"BuddyOne::action_one with {arg}")
        self._buddy.react_two(arg)
        
    def react_one(self, arg):
        print(f"BuddyOne::react_one with {arg}")
        

class BuddyTwo:
    def __init__(self):
        self._buddy: Optional[BuddyOne] = None
        
    def set_buddy(self, buddy: BuddyOne):
        self._buddy = buddy
        
    def action_two(self, arg):
        print(f"BuddyTwo::action_two with {arg}")
        self._buddy.react_one(arg)
        
    def react_two(self, arg):
        print(f"BuddyTwo::react_two with {arg}")

In [5]:
b1 = BuddyOne()
b2 = BuddyTwo()
b1.set_buddy(b2)
b2.set_buddy(b1)

b1.action_one("hello")
b2.action_two("world")

BuddyOne::action_one with hello
BuddyTwo::react_two with hello
BuddyTwo::action_two with world
BuddyOne::react_one with world


## Solution
First slap a new interface on top of the tightly coupled components. This interface should have methods to get the registered `Mediator` and get/set the component's state. Now wherever one component was calling the other, replace that part of the code so that it gets the `Mediator` from its parent and calls the `mediate` method on it passing itself so that the mediator can identify the caller.

The concrete implementation of the `Mediator` will know the interaction between the components. Of course the concrete mediator can quickly get out of hand.

In [8]:
from abc import ABC, abstractmethod

class Mediator(ABC):
    @abstractmethod
    def mediate(self, buddy: "Buddy"):
        pass
    
class Buddy(ABC):
    def __init__(self, mediator: Mediator):
        self._mediator: "Mediator" = mediator
            
    @property
    def mediator(self):
        return self._mediator
    
    @property
    @abstractmethod
    def state(self):
        pass
    
    @state.setter
    @abstractmethod
    def state(self, arg):
        pass


class BuddyOne(Buddy):
    def __init__(self, mediator: "Mediator"):
        super().__init__(mediator)
        self._state = None
        
    @property
    def state(self):
        return self._state
    
    @state.setter
    def state(self, arg):
        self._state = arg
        
    def action_one(self, arg):
        print(f"BuddyOne::action_one with {arg}")
        self.state = arg
        self.mediator.mediate(self)
        
    def react_one(self, arg):
        print(f"BuddyOne::react_one with {arg}")
        
        
class BuddyTwo(Buddy):
    def __init__(self, mediator: "Mediator"):
        super().__init__(mediator)
        self._state = None
        
    @property
    def state(self):
        return self._state
    
    @state.setter
    def state(self, arg):
        self._state = arg
        
    def action_two(self, arg):
        print(f"BuddyTwo::action_two with {arg}")
        self.state = arg
        self.mediator.mediate(self)
        
    def react_two(self, arg):
        print(f"BuddyTwo::react_two with {arg}")
    
    
class ConcreteMediator(Mediator):
    def __init__(self):
        self._buddy1 = None
        self._buddy2 = None
        
    def set_buddies(self, b1, b2):
        self._buddy1 = b1
        self._buddy2 = b2
        
    def mediate(self, buddy: Buddy):
        arg = buddy.state
        if buddy == self._buddy1:
            self._buddy2.react_two(arg)
        elif buddy == self._buddy2:
            self._buddy1.react_one(arg)

In [10]:
mediator = ConcreteMediator()
b1 = BuddyOne(mediator)
b2 = BuddyTwo(mediator)
mediator.set_buddies(b1, b2)

b1.action_one("hello")
b2.action_two("world")

BuddyOne::action_one with hello
BuddyTwo::react_two with hello
BuddyTwo::action_two with world
BuddyOne::react_one with world
